In [1]:
import torch

In [2]:
x=torch.zeros(3,3)
x[0][1]=1

In [3]:
print(x)

tensor([[0., 1., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [2]:
x=torch.rand(3,3)
print(x.requires_grad)

False


In [3]:
x.requires_grad = True
y = torch.rand(3,3,requires_grad=True)
t=x+2*y

In [4]:
t

tensor([[1.1873, 2.2245, 1.2833],
        [1.6637, 1.6832, 2.6307],
        [2.0805, 1.7799, 1.2985]], grad_fn=<AddBackward0>)

In [6]:
z = t.sum()

In [7]:
z.backward()

In [8]:
y.grad

tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])

In [18]:
x.grad

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [19]:
y.grad

tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])

In [21]:
torch.__version__

'1.8.0'

In [2]:
from torch import nn

In [3]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(1,20),
            nn.ReLU(),
            nn.Linear(20, 10),
            nn.ReLU(),
            nn.Linear(10, 1),
        )

    def forward(self, x):
        x = self.linear_relu_stack(x)
        return x

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [36]:
model = SimpleNet().to(device)
print(model)

SimpleNet(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=1, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [6]:
def f1(x):
    return f2(x)+3

def f2(x):
    return x+5

In [7]:
print(f1(10))

18


Python supports "caller definition before callee definition", too

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
import numpy as np

In [22]:
x_values = [i for i in range(10000)]
y_values = [2*i+5 for i in range(10000)]
x_train = np.array(x_values,dtype=np.float32)
y_train = np.array(y_values,dtype=np.float32)
x_train = x_train.reshape(-1,1)
y_train = y_train.reshape(-1,1)

In [23]:
inputs = torch.from_numpy(x_train)
labels = torch.from_numpy(y_train)

In [24]:
inputs = inputs.cuda()
labels = labels.cuda()

In [34]:
labels

tensor([[5.0000e+00],
        [7.0000e+00],
        [9.0000e+00],
        ...,
        [1.9999e+04],
        [2.0001e+04],
        [2.0003e+04]], device='cuda:0')

In [41]:
epochs = 100
loss_fn = nn.MSELoss()
learning_rate = 1e-10
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [56]:
for i in range(epochs):
    pred = model(inputs)
    loss = loss_fn(pred,labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if i % 10 == 0:
            loss = loss.item()
            print(f"loss: {loss} ")

loss: 20.382068634033203 
loss: 20.495601654052734 
loss: 35.64656066894531 
loss: 20.133028030395508 
loss: 20.246835708618164 
loss: 20.371545791625977 
loss: 20.495601654052734 
loss: 35.673675537109375 
loss: 20.126237869262695 
loss: 20.235137939453125 


In [57]:
model(torch.tensor([10.0],device=device))

tensor([19.4829], device='cuda:0', grad_fn=<AddBackward0>)

In [59]:
model(torch.tensor([5000.0],device=device))

tensor([9999.1650], device='cuda:0', grad_fn=<AddBackward0>)

In [60]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear=nn.Linear(1,1)

    def forward(self, x):
        x = self.linear(x)
        return x

In [61]:
model2 = Net().to(device)

In [64]:
optimizer2 = torch.optim.SGD(model2.parameters(), lr=learning_rate)

In [81]:
for i in range(epochs):
    pred = model2(inputs)
    loss = loss_fn(pred,labels)
    
    optimizer2.zero_grad()
    loss.backward()
    optimizer2.step()
    
    if i % 10 == 0:
            loss = loss.item()
            print(f"loss: {loss} ")

loss: 22.148252487182617 
loss: 22.148252487182617 
loss: 22.148252487182617 
loss: 22.148252487182617 
loss: 22.148252487182617 
loss: 22.148252487182617 
loss: 74.53807067871094 
loss: 74.53807067871094 
loss: 74.53807067871094 
loss: 22.148252487182617 


In [97]:
model2(torch.tensor([30.000000],device=device))

tensor([60.2127], device='cuda:0', grad_fn=<AddBackward0>)

In [86]:
model2

Net(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [93]:
for para in model2.parameters():
    print(para)

Parameter containing:
tensor([[2.0010]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.2127], device='cuda:0', requires_grad=True)


In [88]:
2.0010*15000+0.2127

30015.2127

In [116]:
30*2.0005+0.2127

60.227700000000006

In [98]:
model3 = Net()

In [99]:
for para in model3.parameters():
    print(para)

Parameter containing:
tensor([[0.1730]], requires_grad=True)
Parameter containing:
tensor([0.5592], requires_grad=True)


In [112]:
model3(torch.tensor([300000.000000]))

tensor([51912.2812], grad_fn=<AddBackward0>)

In [114]:
0.173039*300000+0.5592

51912.2592

In [117]:
inputs2 = x_train[:10]
labels2 = y_train[:10]

In [118]:
inputs2 = torch.from_numpy(inputs2)
labels2 = torch.from_numpy(labels2)

In [119]:
optimizer3 = torch.optim.SGD(model3.parameters(), lr=1e-5)

In [228]:
for i in range(100000):
    pred = model3(inputs2)
    loss = loss_fn(pred,labels2)
    
    optimizer3.zero_grad()
    loss.backward()
    optimizer3.step()
    
    if i % 10000 == 0:
            loss = loss.item()
            print(f"loss: {loss} ")

loss: 0.0008488130988553166 
loss: 0.0007060206262394786 
loss: 0.0005763749359175563 
loss: 0.0005727015668526292 
loss: 0.0005727015668526292 
loss: 0.0005727015668526292 
loss: 0.0005727015668526292 
loss: 0.0005727015668526292 
loss: 0.0005727015668526292 
loss: 0.0005727015668526292 


In [231]:
model3(torch.tensor([7.0]))

tensor([19.0061], grad_fn=<AddBackward0>)

In [232]:
for para in model3.parameters():
    print(para)

Parameter containing:
tensor([[2.0072]], requires_grad=True)
Parameter containing:
tensor([4.9556], requires_grad=True)


In [255]:
optimizer4 = torch.optim.SGD(model2.parameters(), lr=1e-7)

In [256]:
for i in range(100000):
    pred = model2(inputs[:10])
    loss = loss_fn(pred,labels[:10])
    
    optimizer4.zero_grad()
    loss.backward()
    optimizer4.step()
    
    if i % 10000 == 0:
            loss = loss.item()
            print(f"loss: {loss} ")

loss: 0.0004253497754689306 
loss: 0.0004253497754689306 
loss: 0.0004253497754689306 
loss: 0.0004253497754689306 
loss: 0.0004253497754689306 
loss: 0.0004253497754689306 
loss: 0.0004253497754689306 
loss: 0.0004253497754689306 
loss: 0.0004253497754689306 
loss: 0.0004253497754689306 


In [257]:
for para in model2.parameters():
    print(para)

Parameter containing:
tensor([[2.0068]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([4.9617], device='cuda:0', requires_grad=True)


In [263]:
model2(torch.tensor([7.0],device=device))

tensor([19.0027], device='cuda:0', grad_fn=<AddBackward0>)

In [266]:
model(torch.tensor([7.0],device=device))

tensor([13.5021], device='cuda:0', grad_fn=<AddBackward0>)

In [264]:
for para in model.parameters():
    print(para)

Parameter containing:
tensor([[ 0.0898],
        [ 1.0134],
        [-0.5970],
        [-0.8821],
        [ 0.0586],
        [ 0.8664],
        [ 0.2705],
        [-0.1725],
        [ 0.8910],
        [ 0.8676],
        [-0.0288],
        [-0.0254],
        [-0.7190],
        [ 0.8195],
        [ 0.4827],
        [ 0.9346],
        [-0.9136],
        [ 0.5438],
        [-0.7012],
        [ 0.7345]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.0563,  0.1388,  0.0828, -0.4427, -0.7669, -0.0726,  0.6208, -0.8709,
         0.6948, -0.6680,  0.1063,  0.4701,  0.9915, -0.0704,  0.7291, -0.6851,
         0.9482, -0.3847, -0.8657,  0.7386], device='cuda:0',
       requires_grad=True)
Parameter containing:
tensor([[ 0.1651,  0.2446,  0.2232, -0.2217, -0.1120,  0.2216, -0.0637, -0.2230,
          0.2353, -0.0814, -0.2078, -0.1076,  0.0460,  0.1750,  0.0428,  0.2284,
         -0.0022, -0.0656, -0.0921,  0.1247],
        [ 0.1695,  0.0272, -0.0497, -0.1869, -0.1135, -0.08

In [267]:
import torchvision.models as models

In [268]:
model_vgg = models.vgg16(pretrained=True)
torch.save(model.state_dict(), 'model_weights.pth')

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to C:\Users\hcl90/.cache\torch\hub\checkpoints\vgg16-397923af.pth
100.0%


In [270]:
torch.save(model2,"./data/model2")

In [271]:
model_best = torch.load('./data/model2')

In [272]:
model_best(torch.tensor([7.0],device=device))

tensor([19.0027], device='cuda:0', grad_fn=<AddBackward0>)

In [273]:
model_best.eval()

Net(
  (linear): Linear(in_features=1, out_features=1, bias=True)
)

In [281]:
model_best(torch.tensor([1000.0],device=device))

RuntimeError: CUDA error: unknown error

In [280]:
2.0067*1000+4.9617

2011.6616999999999